In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import nltk
import re
import inspect
import pymorphy3
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [57]:
administrative = pd.read_csv('./administrative.csv', sep=';')
bankruptcy = pd.read_csv('./bankruptcy.csv', sep=';')
civile = pd.read_csv('./civile.csv', sep=';')

### Проверяем нет ли дубликатов

In [10]:
print(administrative.shape)
print(bankruptcy.shape)
print(civile.shape)

(359, 2)
(771, 2)
(1347, 2)


In [11]:
civile.head()

,category,text
0,Гражданское,акционерный общество газпром межрегионгаз каза...
1,Гражданское,индивидуальный предприниматель фио далее предп...
2,Гражданское,общество ограниченный ответственность экострой...
3,Гражданское,общество ограниченный ответственность технопро...
4,Гражданское,акционерный общество семеноводческий хозяйство...


In [12]:
len(administrative.value_counts()), len(bankruptcy.value_counts()), len(civile.value_counts())

(359, 770, 1345)

### Удаляем дубликаты

In [13]:
civile = civile.drop_duplicates()
print(civile.shape)
bankruptcy = bankruptcy.drop_duplicates()
print(bankruptcy.shape)
administrative = administrative.drop_duplicates()
print(administrative.shape)

(1345, 2)
(770, 2)
(359, 2)


In [14]:
data = []
for i, row in pd.concat([civile, bankruptcy, administrative]).iterrows():
    data.append({'text': row['text'], 'category': row['category']})
random.shuffle(data)

In [20]:
len(data)

2474

In [16]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')
months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',]
all_stop_words = stop_words + months
word_tokenizer = nltk.WordPunctTokenizer()

regex = re.compile(r'[А-Яа-яёЁ-]+')
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

def process_data(data):
    texts = []
    targets = []

    for item in tqdm(data):

        text_lower = words_only(item['text'])
        tokens     = word_tokenizer.tokenize(text_lower)

        tokens = [word for word in tokens if (word not in all_stop_words and not word.isnumeric())]

        texts.append(tokens)

    return texts

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/20223210/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Проводим предобработку

In [17]:
y = [item['category'] for item in data]
texts = process_data(data)

  0%|          | 0/2474 [00:00<?, ?it/s]

In [62]:
print(len(y))
print(len(texts))

2474
2474


### Нормализуем и записываем в файл

In [22]:
morph = pymorphy3.MorphAnalyzer()

In [23]:
with open('lemmatized_possed.txt', 'a') as file:
    for i in tqdm(range(len(texts))):
        solution_lemma = [f'{morph.parse(x)[0].normal_form}_{morph.parse(x)[0].tag.POS}' for x in texts[i]]
        solution = ' '.join(solution_lemma)
        file.write(solution + '\n')

  0%|          | 0/2474 [00:00<?, ?it/s]

### Конвертируем pos тэги в используемые моделями rusvectores

In [24]:
openCorpora2Universal = {
    'NOUN': 'NOUN',
    'ADJF': 'ADJ',
    'ADJS': 'ADJ',
    'VERB': 'VERB',
    'INFN': 'VERB',
    'PRTF': 'VERB',
    'PRTS': 'VERB',
    'PRTS': 'VERB',
    'GRND': 'VERB',
    'NUMR': 'NUM',
    'ADVB': 'ADV',
    'NPRO': 'PRON',
    'CONJ': 'CCONJ',
    'PRCL': 'PART'
}
with open('lemmatized_possed.txt', 'r') as file:
    for line in tqdm(file):
        conved = []
        for w in line.split(' '):
            wl = w.split('_')
            try: 
                if wl[1] != None and wl[1] != 'None':
                    conved.append('_'.join([wl[0], openCorpora2Universal[wl[1]]]))
            except:
                pass
        with open('lemmatized_possed_conved.txt', 'a') as fileconved:
                    fileconved.write(f"{' '.join(conved)} \n") 

0it [00:00, ?it/s]

In [25]:
solutions = [x.replace('\n','') for x in open('lemmatized_possed_conved.txt', encoding = 'utf-8').readlines()]

In [26]:
len(solutions)

2474

In [27]:
train_texts, test_texts, train_y, test_y = train_test_split(solutions, y, test_size=0.2, random_state=42, stratify = y)

### Tfid

In [28]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=300,
    norm = None
)

word_vectorizer.fit(solutions)

TfidfVectorizer(max_features=300, ngram_range=(1, 4), norm=None,
                sublinear_tf=True, token_pattern='\\w{1,}')

In [52]:
print(len(train_texts))
print(len(train_y))

1979
1979


In [29]:
tfidf_train = word_vectorizer.transform(train_texts)
tfidf_test = word_vectorizer.transform(test_texts)

In [30]:
print('Shape of tfidf_train:', tfidf_train.shape)
print('Shape of tfidf_test:', tfidf_test.shape)

Shape of tfidf_train: (1979, 300)
Shape of tfidf_test: (495, 300)


In [31]:
word_vectorizer.get_feature_names_out()[:10]

array(['административный_adj', 'административный_adj правонарушение_noun',
       'адрес_noun', 'акт_noun', 'апелляционный_adj',
       'апелляционный_adj жалоба_noun', 'апк_noun', 'апк_noun рф_noun',
       'арбитражный_adj', 'арбитражный_adj процессуальный_adj'],
      dtype=object)

### Обучение

In [34]:
train_X = word_vectorizer.fit_transform(train_texts)
test_X  = word_vectorizer.transform(test_texts)
print('Shape of tfidf_train:', train_X.shape)
print('Shape of tfidf_test:', test_X.shape)

Shape of tfidf_train: (1979, 300)
Shape of tfidf_test: (495, 300)


In [46]:
print(train_X.getnnz())

382558


In [36]:
clf = RandomForestClassifier(n_estimators = 500, max_depth = 10)
clf = clf.fit(train_X, train_y)

In [37]:
pred = clf.predict(test_X)

In [38]:
print('Предсказанные метки: ', pred[0:20], ".....")
print('Истинные метки: ', test_y[0:20], ".....")

Предсказанные метки:  ['Гражданское' 'Банкротное' 'Гражданское' 'Гражданское' 'Банкротное'
 'Административное' 'Гражданское' 'Гражданское' 'Административное'
 'Банкротное' 'Гражданское' 'Гражданское' 'Административное' 'Банкротное'
 'Гражданское' 'Административное' 'Гражданское' 'Гражданское'
 'Гражданское' 'Гражданское'] .....
Истинные метки:  ['Гражданское', 'Банкротное', 'Гражданское', 'Гражданское', 'Банкротное', 'Административное', 'Гражданское', 'Гражданское', 'Административное', 'Банкротное', 'Гражданское', 'Гражданское', 'Административное', 'Банкротное', 'Гражданское', 'Административное', 'Гражданское', 'Гражданское', 'Гражданское', 'Гражданское'] .....


### Word2Vec 

In [39]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('220/model.bin', binary=True)

In [68]:
def make_feature_vec(words, model, num_features):
    feature_vec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2key_set = set(model.index_to_key)
    
    words = words.split(' ')
    
    for word in words:
        if word in index2key_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(texts, model, num_features):
    counter = 0
    feature_vecs = np.zeros((len(texts),num_features), dtype='float32')
    
    for text in texts:
        feature_vecs[counter] = make_feature_vec(text, model, num_features)
        counter = counter + 1
    return feature_vecs

In [71]:
train_vecs = get_avg_feature_vecs(train_texts, model, 300)
test_vecs = get_avg_feature_vecs(test_texts, model, 300)

In [72]:
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(train_vecs, train_y)

### Обучение

In [73]:
pred = clf.predict(test_X)

In [74]:
print('Предсказанные метки: ', pred[0:20], ".....")
print('Истинные метки: ', test_y[0:20], ".....")

Предсказанные метки:  ['Гражданское' 'Банкротное' 'Гражданское' 'Гражданское' 'Банкротное'
 'Административное' 'Гражданское' 'Гражданское' 'Административное'
 'Банкротное' 'Гражданское' 'Гражданское' 'Административное' 'Банкротное'
 'Гражданское' 'Административное' 'Гражданское' 'Гражданское'
 'Гражданское' 'Гражданское'] .....
Истинные метки:  ['Гражданское', 'Банкротное', 'Гражданское', 'Гражданское', 'Банкротное', 'Административное', 'Гражданское', 'Гражданское', 'Административное', 'Банкротное', 'Гражданское', 'Гражданское', 'Административное', 'Банкротное', 'Гражданское', 'Административное', 'Гражданское', 'Гражданское', 'Гражданское', 'Гражданское'] .....
